In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import librosa
import librosa.display

import matplotlib.pyplot as plt
import seaborn as sns

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout, GlobalAveragePooling2D 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os

import numpy as np

In [ ]:
spectrogram_path = '/kaggle/input/dl-final-musicgenreclassification-dataset/data/spectrogram'

In [ ]:
labels = os.listdir('/kaggle/input/dl-final-musicgenreclassification-dataset/data/spectrogram/train')

# Get the class names
classes = os.listdir(data_folder)
print(classes)
classes.remove('.DS_Store')
classes.sort()
print(f'Total classes are: {len(classes)}')
print(classes)
# Show the first image in each folder
fig = plt.figure(figsize=(10,6))
i = 0

for sub_dir in classes:
    i+=1
    img_file = os.listdir(os.path.join(data_folder,sub_dir))[0]
    img_path = os.path.join(data_folder, sub_dir, img_file)
    img = mpimg.imread(img_path)
    a = fig.add_subplot(1, len(classes), i)
    a.axis('off')
    imgplot = plt.imshow(img)
    a.set_title(img_file)
plt.show()    

In [8]:
img_size = (256, 128)
batch_size = 10

print("Getting Data...")
datagen = ImageDataGenerator(rescale=1./255, # normalize pixel values
                             validation_split=0.3) # hold back 30% of the images for validation

print("Preparing training dataset...")
train_generator = datagen.flow_from_directory(
    data_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

print("Preparing validation dataset...")
validation_generator = datagen.flow_from_directory(
    data_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

classnames = list(train_generator.class_indices.keys())
print('Data generators ready')

Getting Data...
Preparing training dataset...
Found 661 images belonging to 5 classes.
Preparing validation dataset...
Found 280 images belonging to 5 classes.
Data generators ready


In [9]:
# Define a CNN classifier network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(256, (2, 2), input_shape=train_generator.image_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(128, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Dropout(0.6))
model.add(Flatten())

model.add(Dense(train_generator.num_classes, activation='softmax'))

# With the layers defined, we can now compile the model for categorical (multi-class) classification
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 255, 127, 256)     3328      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 85, 42, 256)      0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 84, 41, 128)       131200    
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 28, 13, 128)      0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 27, 12, 64)        32832     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 9, 4, 64)        

In [10]:
# Train the model over 5 epochs using 30-image batches and using the validation holdout dataset for validation
num_epochs = 15
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = num_epochs)
history = model.fit(train_generator,validation_generator,batch_size=16,epochs=80)

Epoch 1/15
66/66 [==============================] - 21s 315ms/step - loss: 1.6235 - accuracy: 0.2028 - val_loss: 1.6063 - val_accuracy: 0.2357
Epoch 2/15
66/66 [==============================] - 21s 320ms/step - loss: 1.6130 - accuracy: 0.2212 - val_loss: 1.6043 - val_accuracy: 0.2107
Epoch 3/15
66/66 [==============================] - 23s 352ms/step - loss: 1.6061 - accuracy: 0.2089 - val_loss: 1.6011 - val_accuracy: 0.2143
Epoch 4/15
66/66 [==============================] - 24s 364ms/step - loss: 1.5996 - accuracy: 0.2565 - val_loss: 1.5960 - val_accuracy: 0.2607
Epoch 5/15
66/66 [==============================] - 23s 342ms/step - loss: 1.5809 - accuracy: 0.2657 - val_loss: 1.5780 - val_accuracy: 0.2714
Epoch 6/15
66/66 [==============================] - 22s 339ms/step - loss: 1.5692 - accuracy: 0.2611 - val_loss: 1.5769 - val_accuracy: 0.2536
Epoch 7/15
66/66 [==============================] - 22s 329ms/step - loss: 1.5400 - accuracy: 0.3287 - val_loss: 1.5892 - val_accuracy: 0.2893

ValueError: `y` argument is not supported when using `keras.utils.Sequence` as input.

In [ ]:
epoch_nums = range(1,num_epochs+1)
training_loss = history.history["loss"]
validation_loss = history.history["val_loss"]
plt.figure(figsize=(10,6))
plt.plot(epoch_nums, training_loss)
plt.plot(epoch_nums, validation_loss)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Tensorflow doesn't have a built-in confusion matrix metric, so we'll use SciKit-Learn

print("Generating predictions from validation data...")
# Get the image and label arrays for the first batch of validation data
x_test = validation_generator[0][0]
y_test = validation_generator[0][1]

# Use the model to predict the class
class_probabilities = model.predict(x_test)

# The model returns a probability value for each class
# The one with the highest probability is the predicted class
predictions = np.argmax(class_probabilities, axis=1)

# The actual labels are hot encoded (e.g. [0 1 0], so get the one with the value 1
true_labels = np.argmax(y_test, axis=1)

# Plot the confusion matrix
cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(10,6))
plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
plt.colorbar()
tick_marks = np.arange(len(classnames))
plt.xticks(tick_marks, classnames, rotation=85)
plt.yticks(tick_marks, classnames)
plt.xlabel("Actual Shape")
plt.ylabel("Predicted Shape")
plt.show()

In [ ]:
# Save the trained model
modelFileName = 'song_classifier.pb'
model.save(modelFileName)
del model  # deletes the existing model variable
print('model saved as', modelFileName) 

In [ ]:
from tensorflow.keras import models
import numpy as np
from random import randint

# Function to predict the class of an image
def predict_image(classifier, image):
    from tensorflow import convert_to_tensor
    # The model expects a batch of images as input, so we'll create an array of 1 image
    imgfeatures = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])

    # We need to format the input to match the training data
    # The generator loaded the values as floating point numbers
    # and normalized the pixel values, so...
    imgfeatures = imgfeatures.astype('float32')
    imgfeatures /= 255
    
    # Use the model to predict the image class
    class_probabilities = classifier.predict(imgfeatures)
    
    # Find the class predictions with the highest predicted probability
    index = int(np.argmax(class_probabilities, axis=1)[0])
    return index

# Create a random test image
classnames = os.listdir(os.path.join('/Users/tharindudamruwan/Desktop/ML and AI for DS/Intellihack/Songs', 'Spect'))
classnames.sort()
img = mpimg.imread("/Users/tharindudamruwan/Desktop/ML and AI for DS/Intellihack/Songs/Spect/Calm/3.png")
plt.axis('off')
plt.imshow(img)

# Use the classifier to predict the class
model = models.load_model('song_classifier.h5') # loads the saved model
class_idx = predict_image(model, img)
print (classnames[class_idx])